# Test note for utpakkingsfunksjon i Utils.py

In [ ]:
%run ../../SSB_Spark_tools/data_processing/Utils.py

In [ ]:
import pyspark.sql.functions as F

In [ ]:
def tverrsnitt(df, coDate=None):
    if coDate!=None:
        df = df.filter(F.col('hendelsetidspunkt') <= coDate)
    df_dato = df.join(df.groupBy('identifikator', 'gjelderPeriode').agg(F.max('hendelsetidspunkt').alias("hendelsetidspunkt")), ['identifikator', 'gjelderPeriode', 'hendelsetidspunkt'], how='inner')
    return df_dato

In [ ]:
skatt_raw = spark.read.path("/kilde/ske/skatt/person/fastsatt/rådata/2019v1.0.1")

In [ ]:
skattemelding_hendelse = skatt_raw\
                        .select('hendelse.*', 'skattemeldingUtflatet')\
                        .withColumnRenamed('registreringstidspunkt', 'hendelsetidspunkt')\
                        .withColumn('hendelsetidspunkt', F.to_timestamp('hendelsetidspunkt', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
skattemelding_utflatet = skattemelding_hendelse\
                        .select('identifikator', 'gjelderPeriode', 'hendelsetidspunkt', 'hendelsetype', 'skattemeldingUtflatet.*')\
                        .withColumn('registreringstidspunkt', F.to_timestamp('registreringstidspunkt', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))\
                        .cache()
                        
skattemelding_tverrsnitt =  tverrsnitt(skattemelding_utflatet).cache()

tverrsnitt_filtrert = skattemelding_tverrsnitt\
                        .filter(F.col('hendelsetype')!='slettet')\
                        .filter(F.col('skjermet')==False)\
                        .filter(F.col('identifikator')==F.col('personidentifikator'))\
                        .cache()

In [ ]:
so_dict_testutpakking = unpack_parquet(tverrsnitt_filtrert, rootdf=True, rootvar=['personidentifikator'])
so_corevar = {}
for k in so_dict_testutpakking.keys():
    so_corevar[k]= list(so_dict_testutpakking[k].columns)
    print(k)